In [1]:
from glob import glob

for g in glob(r'C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap13\2025학년도 서울 학교운동부 운영 계획(매뉴얼)_공문 발송용.pdf'):
    print(g)

C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap13\2025학년도 서울 학교운동부 운영 계획(매뉴얼)_공문 발송용.pdf


In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def read_pdf_and_split_text(pdf_path, chunk_size=300, chunk_overlap=100):
    """
    주어진 PDF 파일을 읽고 텍스트를 분할합니다.
    매개변수:
        pdf_path (str): PDF 파일의 경로.
        chunk_size (int, 선택적): 각 텍스트 청크의 크기, 기본값은 300입니다.
        chunk_overlap (int, 선택적): 청크 간의 중첩 크기, 기본값은 100입니다.
    반환값:
        list: 분할된 텍스트 청크의 리스트
    """
    pdf_path= r'C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap13\2025학년도 서울 학교운동부 운영 계획(매뉴얼)_공문 발송용.pdf'
    
    print(f"PDF: {pdf_path} -------------------------")
    pdf_loader = PyPDFLoader(pdf_path)
    data_from_pdf = pdf_loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size, chunk_overlap = chunk_overlap
    )

    splits = text_splitter.split_documents(data_from_pdf)

    print(f"Number of splits:{len(splits)}\n")
    return splits

In [3]:
# [VS Code / Codespaces] Gemini API (text-embedding-004 + Robust Retry)
import os
import time
from glob import glob
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# 1. 환경 변수 로드
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY가 설정되지 않았습니다.")

# 2. 최신 임베딩 모델 설정 (text-embedding-004)
embedding = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004", 
    google_api_key=GOOGLE_API_KEY
)

# 3. 저장 경로 설정 (모델이 바뀌었으므로 경로도 구분하는 것이 좋습니다)
persist_directory = './chroma_store_gemini_seoul'

# 4. 벡터 스토어 생성 로직
if os.path.exists(persist_directory):
    print(f"Loading existing Chroma store from {persist_directory}")
    vectorstore = Chroma(
        persist_directory=persist_directory, 
        embedding_function=embedding
    )
else:
    print("Creating new Chroma store...")
    vectorstore = None
    
    # PDF 파일 경로 설정 (본인의 경로에 맞게 수정)
    pdf_files = glob(r'C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap13\2025학년도 서울 학교운동부 운영 계획(매뉴얼)_공문 발송용.pdf') 
    
    for g in pdf_files:
        print(f"Processing file: {g}")
        # read_pdf_and_split_text 함수는 이미 정의되어 있다고 가정
        chunks = read_pdf_and_split_text(g)
        
        # [중요] Rate Limit 방지를 위한 설정
        # 무료 티어는 분당 요청(RPM) 제한이 있으므로 한 번에 조금씩 처리합니다.
        batch_size = 10 
        
        for i in range(0, len(chunks), batch_size):
            batch = chunks[i:i+batch_size]
            
            # 재시도(Retry) 로직: 에러 발생 시 대기 후 재시도
            max_retries = 5
            for attempt in range(max_retries):
                try:
                    if vectorstore is None:
                        vectorstore = Chroma.from_documents(
                            documents=batch,
                            embedding=embedding,
                            persist_directory=persist_directory
                        )
                    else:
                        vectorstore.add_documents(documents=batch)
                    
                    print(f"  - Processed batch {i} ~ {i+len(batch)}")
                    
                    # [핵심] 성공 후 3초 대기 (분당 15회 제한 준수)
                    time.sleep(3) 
                    break  # 성공하면 재시도 루프 탈출
                    
                except Exception as e:
                    # 429(ResourceExhausted) 에러 발생 시
                    if "429" in str(e) or "ResourceExhausted" in str(e):
                        wait_time = (attempt + 1) * 30 # 30초, 60초... 점진적 대기
                        print(f"  ⚠️ Quota exceeded. Waiting {wait_time} seconds...")
                        time.sleep(wait_time)
                    else:
                        print(f"  ❌ Error: {e}")
                        raise e # 다른 에러는 즉시 중단

Creating new Chroma store...
Processing file: C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap13\2025학년도 서울 학교운동부 운영 계획(매뉴얼)_공문 발송용.pdf
PDF: C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap13\2025학년도 서울 학교운동부 운영 계획(매뉴얼)_공문 발송용.pdf -------------------------
Number of splits:1401

  - Processed batch 0 ~ 10
  - Processed batch 10 ~ 20
  - Processed batch 20 ~ 30
  - Processed batch 30 ~ 40
  - Processed batch 40 ~ 50
  - Processed batch 50 ~ 60
  - Processed batch 60 ~ 70
  - Processed batch 70 ~ 80
  - Processed batch 80 ~ 90
  - Processed batch 90 ~ 100
  - Processed batch 100 ~ 110
  - Processed batch 110 ~ 120
  - Processed batch 120 ~ 130
  - Processed batch 130 ~ 140
  - Processed batch 140 ~ 150
  - Processed batch 150 ~ 160
  - Processed batch 160 ~ 170
  - Processed batch 170 ~ 180
  - Processed batch 180 ~ 190
  - Processed batch 190 ~ 200
  - Processed batch 200 ~ 210
  - Processed batch 210 ~ 220
  - Processed batch 220 ~ 230
  - Processe

In [4]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

chunks = retriever.invoke("운동부 전지훈련 계획 작성?")

for chunk in chunks:
    print(chunk.metadata)
    print(chunk.page_content)

{'creator': 'Adobe InDesign 20.0 (Windows)', 'trapped': '/False', 'page_label': '40', 'page': 39, 'creationdate': '2025-01-23T14:24:05+09:00', 'source': 'C:\\Users\\ziclp\\OneDrive\\Desktop\\LLM Assignment\\1week\\PRACTICE1\\chap13\\2025학년도 서울 학교운동부 운영 계획(매뉴얼)_공문 발송용.pdf', 'moddate': '2025-01-23T14:43:00+09:00', 'total_pages': 264, 'producer': 'Adobe PDF Library 17.0'}
지도가 가능한 수준인가?
현지사정 •  전지훈련 장소에는 특별히 고려해야 할 지역적 사안이 있는가?  
해당 사안은 학생들에게 어떤  영향을 미치며, 학생들은 어떻게 보호할 수 있는가?
방역지침 •  감염병 확산 등의 국가적(지역적)인 방역지침은 준수하는 것이 가능한가?
불참관련 •  전지훈련에 참여하지 않으려는 학생에게 비용을 전가하거나, 불참을 이유로  
부당한 처우를 하는 일은 없는가? 잔류 학생의 지도계획은 무엇인가?
혹서(한)기 훈련 안전대책 수립 및 훈련시간 최소화
{'producer': 'Adobe PDF Library 17.0', 'source': 'C:\\Users\\ziclp\\OneDrive\\Desktop\\LLM Assignment\\1week\\PRACTICE1\\chap13\\2025학년도 서울 학교운동부 운영 계획(매뉴얼)_공문 발송용.pdf', 'creationdate': '2025-01-23T14:24:05+09:00', 'moddate': '2025-01-23T14:43:00+09:00', 'creator': 'Adobe InDesign 20.0 (Windows)', 'trapped': '/False', 'page': 225, 'total_pages': 264, 'page_

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", # 원본 모델 유지
    api_key=GOOGLE_API_KEY,
    temperature=0.3
)
chat.invoke('운동부 전지훈련 계획 작성방법 알려줘')


AIMessage(content='운동부 전지훈련 계획을 작성하는 것은 성공적인 훈련과 선수들의 성장을 위해 매우 중요합니다. 체계적인 계획은 훈련의 효율성을 높이고, 예상치 못한 문제 발생 시 신속하게 대응할 수 있도록 돕습니다.\n\n다음은 전지훈련 계획을 작성하는 단계별 방법과 포함되어야 할 주요 내용입니다.\n\n---\n\n### **운동부 전지훈련 계획 작성 방법**\n\n**I. 계획의 개요 (Overview)**\n\n*   **1. 계획서 제목:**\n    *   예: "2024년 [종목] 운동부 [지역] 전지훈련 계획서"\n*   **2. 훈련 목표:**\n    *   가장 중요합니다. 구체적이고 측정 가능하며 달성 가능한 목표를 설정해야 합니다.\n    *   **예시:**\n        *   **체력 증진:** 기초 체력 향상 (근력, 지구력, 순발력), 부상 방지.\n        *   **기술 향상:** 개인별 약점 보완, 특정 기술 숙련도 향상.\n        *   **전술 이해:** 팀 전술 이해도 증진, 실전 적용 능력 향상.\n        *   **정신력 강화:** 집중력, 인내심, 승부욕 향상, 위기 관리 능력 배양.\n        *   **팀워크 강화:** 동료애 증진, 소통 능력 향상, 리더십 함양.\n        *   **경기력 향상:** 다가오는 대회(예: 전국체전, 리그) 대비 실전 감각 극대화.\n*   **3. 훈련 기간:**\n    *   시작일 ~ 종료일 (총 며칠)\n*   **4. 훈련 장소:**\n    *   훈련 시설 (운동장, 체육관, 수영장 등), 숙소, 식당 등 구체적인 장소 명시.\n    *   장소 선정 이유 (예: 기후, 시설, 접근성 등)\n*   **5. 참가 대상:**\n    *   선수 명단 (학년, 포지션 등), 코치진, 기타 지원 인력 (의무 트레이너, 영양사 등).\n\n---\n\n**II. 훈련 프로그램 (Training Program)**\n\

In [6]:
# Router 설정
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal # 문자열 리터럴 타입을 지원하는 typing 모듈의 클래스
from pydantic import BaseModel, Field

# Data model
class RouteQuery(BaseModel):
    """사용자 쿼리를 가장 관련성이 높은 데이터 소스로 라우팅합니다."""
    
    datasource: Literal["vectorstore", "casual_talk"] = Field(
        ...,
        description="""
        사용자 질문에 따라 casual_talk 또는 vectorstore로 라우팅합니다.
        - casual_talk: 일상 대화를 위한 데이터 소스. 사용자가 일상적인 질문을 할 때 사용합니다.
        - vectorstore: 사용자 질문에 답하기 위해 RAG로 vectorstore 검색이 필요한 경우 사용합니다.
        """,
    )

In [7]:
# 특정 모델을 structured output (구조화된 출력)과 함께 사용하기 위해 설정
structured_llm_router = chat.with_structured_output(RouteQuery)

router_system = """
당신은 사용자의 질문을 vectorstore 또는 casual_talk으로 라우팅하는 전문가입니다.
- vectorstore에는 인공지능을 활용한 서울특별시 학교 운동부 운영과 관련된 문서가 포함되어 있습니다. 이 주제에 대한 질문에는 vectorstore를 사용하십시오.
- 사용자의 질문이 일상 대화에 관련된 경우 casual_talk을 사용하십시오.
"""

# 시스템 메시지와 사용자의 질문을 포함하는 프롬프트 템플릿 생성
route_prompt = ChatPromptTemplate.from_messages([
    ("system", router_system),
    ("human", "{question}"),
])

# 라우터 프롬프트와 구조화된 출력 모델을 결합한 객체
question_router = route_prompt | structured_llm_router


In [8]:
# ✅ 올바른 호출 방식 (.invoke 사용)
print(
    question_router.invoke({
        "question": "운동부 전지훈련 계획 수립 알려줘?"
    })
)

print(question_router.invoke({"question": "잘 지냈어?"}))

datasource='vectorstore'
datasource='casual_talk'


In [9]:
from langchain_core.prompts import PromptTemplate

class GradeDocuments(BaseModel):
    """검색된 문서가 질문과 관련성 있는지 yes 또는 no로 평가합니다."""

    binary_score: Literal["yes", "no"] = Field(
        description="문서가 질문과 관련이 있는지 여부를 'yes' 또는 'no'로 평가합니다."
    )

structured_llm_grader = chat.with_structured_output(GradeDocuments)

In [10]:
grader_prompt = PromptTemplate.from_template("""
당신은 검색된 문서가 사용자 질문과 관련이 있는지 평가하는 평가자입니다. \n 
문서에 사용자 질문과 관련된 키워드 또는 의미가 포함되어 있으면, 해당 문서를 관련성이 있다고 평가하십시오. \n
엄격한 테스트가 필요하지 않습니다. 목표는 잘못된 검색 결과를 걸러내는 것입니다. \n
문서가 질문과 관련이 있는지 여부를 나타내기 위해 'yes' 또는 'no'로 이진 점수를 부여하십시오.
                                             
Retrieved document: \n {document} \n\n 
User question: {question}
""")

retrieval_grader = grader_prompt | structured_llm_grader
question = "운동부 전지훈련 계획 수립 알려줘"
documents = retriever.invoke(question)

for doc in documents:
    print(doc)


page_content='정보가 일괄적으로 저장됨.
   [학교운동부 지도교사·지도자 관리] 화면에서 출력하려는 지도교사·지도자를 선택하고 {출력} 버튼을 클릭하면 ‘학교운동부 
지도자 현황 레포트’ 화면이 호출됨.
[학교운동부 지도자 현황 레포트]' metadata={'producer': 'Adobe PDF Library 17.0', 'source': 'C:\\Users\\ziclp\\OneDrive\\Desktop\\LLM Assignment\\1week\\PRACTICE1\\chap13\\2025학년도 서울 학교운동부 운영 계획(매뉴얼)_공문 발송용.pdf', 'moddate': '2025-01-23T14:43:00+09:00', 'page': 225, 'trapped': '/False', 'total_pages': 264, 'page_label': '226', 'creator': 'Adobe InDesign 20.0 (Windows)', 'creationdate': '2025-01-23T14:24:05+09:00'}
page_content='정보가 삭제됨.
   [학교운동부 지도교사·지도자 관리] 화면에서 일괄적으로 연수 정보를 등록할 지도교사·지도자를 선택하고 
{연수일괄등록} 버튼을 클릭하면 ‘학교운동부 지도교수·지도자 연수 관리 –일괄 등록’ 팝업 화면이 호출됨.' metadata={'producer': 'Adobe PDF Library 17.0', 'moddate': '2025-01-23T14:43:00+09:00', 'creationdate': '2025-01-23T14:24:05+09:00', 'source': 'C:\\Users\\ziclp\\OneDrive\\Desktop\\LLM Assignment\\1week\\PRACTICE1\\chap13\\2025학년도 서울 학교운동부 운영 계획(매뉴얼)_공문 발송용.pdf', 'creator': 'Adobe InDesign 20.0 (Windows)', 'trapped': '/False', 

In [11]:
import time # 1. 시간 지연 기능을 사용하기 위해 import

filtered_docs = []

print(f"검색된 문서 개수: {len(documents)}")

for i, doc in enumerate(documents):
    print(f"Processing Document {i + 1}...")
    
    # [핵심 수정 사항] API 호출 전 강제로 시간을 끕니다.
    # Gemini 무료 티어 한도(분당 10~15회)를 고려하여 7초 대기
    time.sleep(7) 
    
    try:
        # 평가 실행
        is_relevant = retrieval_grader.invoke({"question": question, "document": doc.page_content})
        
        print(f"  - 관련성 평가 결과: {is_relevant.binary_score}")
        print(f"  - 문서 내용 일부: {doc.page_content[:50]}...") # 내용은 너무 길어서 50자만 출력
        print("=================================\n")

        if is_relevant.binary_score == "yes":
            filtered_docs.append(doc)
            
    except Exception as e:
        print(f"  ❌ Error processing document {i+1}: {e}")
        # 에러가 나도 멈추지 않고 다음 문서로 넘어가도록 처리
        continue

print(f"최종 필터링된 문서 개수: {len(filtered_docs)}")

검색된 문서 개수: 20
Processing Document 1...
  - 관련성 평가 결과: no
  - 문서 내용 일부: 정보가 일괄적으로 저장됨.
   [학교운동부 지도교사·지도자 관리] 화면에서 출력하려는 ...

Processing Document 2...
  - 관련성 평가 결과: no
  - 문서 내용 일부: 정보가 삭제됨.
   [학교운동부 지도교사·지도자 관리] 화면에서 일괄적으로 연수 정보를...

Processing Document 3...
  - 관련성 평가 결과: no
  - 문서 내용 일부: 입력한 과목이 추가됨.
❿   [학생선수 학기말성적] 팝업 화면에서 선택한 학생 선수의 최...

Processing Document 4...
  - 관련성 평가 결과: no
  - 문서 내용 일부: ※   관리대상 과목으로 등록된 과목 중 최저학력에 도달하지 못한 과목이 리스트로 조회됨....

Processing Document 5...
  - 관련성 평가 결과: no
  - 문서 내용 일부: ❸   조회 목록에서 {성명}을 클릭하면 ‘학교운동부 지도교사·지도자 상세’화면으로 전환됨...

Processing Document 6...
  - 관련성 평가 결과: no
  - 문서 내용 일부: 출장들을 확인 및 수정한 뒤에 결재할 수 있다.
-   동행자 간의 출장기간이 다른 상태에...

Processing Document 7...
  - 관련성 평가 결과: no
  - 문서 내용 일부: -   출장종류를 국외출장으로 선택하였을 때 나타나는 출장지의 [찾기] 버튼을 클릭하면 국...

Processing Document 8...
  - 관련성 평가 결과: no
  - 문서 내용 일부:    {저장}버튼을 클릭하면 모든 정보를 저장함.
   체크박스를 선택하고 {삭제} 버...

Processing Document 9...
  - 관련성 평가 결과: no
  - 문서 내용 일부: 관리
학교스포츠클럽
관리
학교스포츠클럽종목조회 학교스포츠클럽

In [12]:
### Generate
# PromptTemplate을 사용하여 RAG를 위한 프롬프트를 생성합니다.

rag_generate_system = """
너는 사용자의 질문에 대해 주어진 context에 기반하여 답변하는 학교 운동부 운영 전문가이다. 
주어진 context는 vectorstore에서 검색된 결과이다. 
주어진 context를 기반으로 사용자의 question에 대해 답변하라.

=================================
question: {question}
context: {context}
"""

# PromptTemplate을 생성하여 question과 context를 포맷팅
rag_prompt = PromptTemplate(
    input_variables=["question", "context"],
    template=rag_generate_system
)

# rag chain
rag_chain = rag_prompt | chat 

# 사용자 질문과 검색된 문서를 입력으로 사용하여 RAG를 실행
question = "운동부 전지훈련 계획"

rag_chain.invoke({"question": question, "context": filtered_docs})


AIMessage(content='제공된 context에 따르면, 운동부 전지훈련 계획에 대한 구체적인 내용은 확인할 수 없습니다.\n\n다만, context에는 훈련과 관련된 다음 내용이 언급되어 있습니다.\n*   **훈련 및 상담 기록지** (<서식4>)\n*   **학교운동부 훈련일지 레포트**: 학교운동부 지도교사·지도자가 업무지원(외부서비스)에서 등록 및 저장한 훈련일지 정보가 조회된다고 명시되어 있습니다.\n\n이 정보들은 훈련 기록 및 관리에 관한 것이며, 전지훈련 계획 자체에 대한 내용은 포함하고 있지 않습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--3374ca33-1672-4c77-8615-9f5fce3bf884-0', usage_metadata={'input_tokens': 770, 'output_tokens': 551, 'total_tokens': 1321, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 420}})